# Import Libraries

In [2]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import hashlib
from pinecone import Pinecone
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import json
import ast
from rapidfuzz import fuzz
from datetime import datetime
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain import hub
from langchain_pinecone import PineconeVectorStore

## Initialization

In [ ]:
# Get API Keys
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Pinecone Initialization
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("echo-openai")

retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

docsearch = PineconeVectorStore.from_documents(
    documents="transcript ni siya",
    index_name=input("Give meeting title"),
    embedding="embeddings of transcript",
    namespace=input("Give organization name")
)

retriever = docsearch.as_retriever()

# OpenAI Initialization
client = OpenAI(api_key=OPENAI_API_KEY)
EMBEDDINGS = OpenAIEmbeddings(model='text-embedding-3-small', openai_api_key=OPENAI_API_KEY)
LLM = ChatOpenAI(temperature=0.0, model_name="gpt-4-turbo", openai_api_key=OPENAI_API_KEY)

combine_docs_chain = create_stuff_documents_chain(
    LLM, retrieval_qa_chat_prompt
)

retrieval_chain = create_retrieval_chain(
    retriever, combine_docs_chain
)

C:\Users\Zeck Lowe\AppData\Local\Temp\ipykernel_34708\480772498.py:9: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  client = OpenAI(api_key=OPENAI_API_KEY)


# Query

In [ ]:
query = "What was the QA perspective on the kickoff meeting?"